In [17]:
import numpy as np
from glob import glob
import nibabel as nib
from os.path import join as opj

from nipype.interfaces import spm
from nipype.interfaces import fsl
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.algorithms.misc import Gunzip
from nipype.pipeline.engine import Workflow, Node, MapNode

# MATLAB - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12/toolbox')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

#other study-specific variables
project_home = '/Users/myelin/Dropbox/Projects/TH_NAR_ASL/proc/dev'
subjects_dir = project_home
subjects_list = ['ko']
raw_dir = 'raw'
output_dir = 'proc'

#pre-proc variables
TR = 4.844

#Population specific variables for ASL
nex_asl = 1 #number of excitations from the 3D ASL scan parameters
inversion_efficiency = 0.8 #from GE
background_supp_eff = 0.75 #from GE
efficiency = inversion_efficiency * background_supp_eff 
T1_blood = 1.6 #T1 of blood in seconds(1.6s at 3T and 1.4s at 1.5T)
sat_time = 2 #in seconds, from GE
partition_coeff = 0.9 #whole brain average in ml/g
scaling_factor = 32 #scaling factor, can be taken from PW dicom header at position 0043,107f
postlabel_delay = 2.025 #1.525 #post label delay in seconds
labeling_time = 1.450 #labeling time in seconds
T1_tissue = 1.2 #estimated T1 of grey matter in seconds

In [33]:
# preprocflow nodes

# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['subjid']),
                  name="infosource")
infosource.iterables = [('subjid', subjects_list)]

# SelectFiles
templates = {'pw': subjects_dir + '/raw/{subjid}/pw.nii.gz',
             'pd': subjects_dir + '/raw/{subjid}/pd.nii.gz'}
selectfiles = Node(SelectFiles(templates), name="selectfiles")

#gunzip
gunzip = Node(Gunzip(), name="gunzip")

# Datasink
datasink = Node(DataSink(), name="datasink")
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir

#quantify CBF from PW volume
def quantify_cbf(pw_nifti1,pd_nifti1,sat_time,T1_tissue,postlabel_delay,T1_blood,labeling_time,efficiency,nex_asl,scaling_factor):
    # set variables
    pw_data = pw_nifti1.get_data()
    pd_data = pd_nifti1.get_data()
    conversion = 6000 #to convert values from mL/g/s to mL/100g/min
    PDthresh = 0.5*(np.sum(pd_data**2)/np.sum(pd_data))
    
    cbf_numerator = (1-np.exp(sat_time/T1_tissue))*np.exp(postlabel_delay/T1_blood)
    cbf_denominator = 2*T1_blood*(1-np.exp(labeling_time/T1_blood))*efficiency*nex_asl
    cbf_data = conversion*(cbf_numerator/cbf_denominator)*(pw_data/(scaling_factor*pd_data))
    
    cbf_vol = nib.Nifti1Image(cbf_data, pw_nifti1.affine)
    nib.save(cbf_vol,project_home + '/raw/ko/cbf_vol.nii.gz')

In [34]:
pw_nifti1 = nib.load(project_home + '/raw/ko/pw.nii.gz')
pd_nifti1 = nib.load(project_home + '/raw/ko/pd.nii.gz')
quantify_cbf(pw_nifti1,pd_nifti1,sat_time,T1_tissue,postlabel_delay,T1_blood,labeling_time,efficiency,nex_asl,scaling_factor)


4.0
646673196
73026900


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:46: RuntimeWarning: divide by zero encountered in divide
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:46: RuntimeWarning: invalid value encountered in divide


In [ ]:
aslprocflow = Workflow(name="aslprocflow")

aslprocflow.base_dir = opj(subjects_dir)
aslprocflow.connect([(infosource, selectfiles, [('subjid', 'subjid')]),
                     (selectfiles, gunzip, [('pw', 'in_file')]),
                     (gunzip, 
                    ])
aslprocflow.write_graph(graph2use='flat')
aslprocflow.run('MultiProc', plugin_args={'n_procs': 2})